# 🔍 Routing Setup Variability and Plant Performance Analysis

This notebook investigates how differences in routing setups across plants impact production performance. Specifically, we focus on materials that are:

- Produced in **multiple plants**
- Assigned the **same routing ID**
- Configured with **different setup parameters** (setup time, cycle time, labour time)

By joining routing data with production output, we analyze key performance indicators (KPIs) such as throughput per hour, cycle time variance, and order duration. The goal is to identify whether setup inconsistencies contribute to execution inefficiencies and to highlight opportunities for standardization.


In [0]:
%sql
-- Step 1: Materials produced in multiple plants
WITH multi_plant_output AS (
  SELECT material
  FROM workspace.default.output
  GROUP BY material
  HAVING COUNT(DISTINCT plant) > 1
)
SELECT * FROM multi_plant_output;



material
7839956
7841154
7835272
4840353
7824415
7840677
7840676
7815212
4840355
4802569


In [0]:
%sql
--  Step 2: Materials with same routing ID but different setup across plants
WITH multi_plant_output AS (
  SELECT material
  FROM workspace.default.output
  GROUP BY material
  HAVING COUNT(DISTINCT plant) > 1
),
routing_variants AS (
  SELECT
    r.material_id,
    r.id AS routing_id
  FROM workspace.default.routings r
  JOIN multi_plant_output mpo
    ON r.material_id = mpo.material
  GROUP BY r.material_id, r.id
  HAVING COUNT(DISTINCT r.plant_id) > 1
     AND COUNT(DISTINCT named_struct(
           'setup', r.setup_time_mins,
           'cycle', r.cycle_time_sec,
           'labour', r.labour_time_sec
         )) > 1
)
SELECT * FROM routing_variants;


material_id,routing_id
6226661,3
7835187,1
7823191,1
7823202,3
5783751,1
7823346,1
5781601,1
5781595,1
7835189,1
7839366,1


In [0]:
%sql
--  Step 3: Join routing variants with output to compare performance
WITH multi_plant_output AS (
  SELECT material
  FROM workspace.default.output
  GROUP BY material
  HAVING COUNT(DISTINCT plant) > 1
),
routing_variants AS (
  SELECT
    r.material_id,
    r.id AS routing_id
  FROM workspace.default.routings r
  JOIN multi_plant_output mpo
    ON r.material_id = mpo.material
  GROUP BY r.material_id, r.id
  HAVING COUNT(DISTINCT r.plant_id) > 1
     AND COUNT(DISTINCT named_struct(
           'setup', r.setup_time_mins,
           'cycle', r.cycle_time_sec,
           'labour', r.labour_time_sec
         )) > 1
)

SELECT
  o.material,
  o.production_order,
  o.plant,
  o.line,
  r.id AS routing_id,
  o.linetype,
  o.order_start_ts,
  o.order_end_ts,
  DATEDIFF(o.order_end_ts, o.order_start_ts) AS duration_days,
  o.approved_quantity,
  o.actual_net_production_time,
  ROUND(try_divide(o.approved_quantity, o.actual_net_production_time / 3600), 2) AS throughput_per_hour,
  o.cycletime AS actual_cycle_time,
  r.cycle_time_sec AS planned_cycle_time,
  ROUND(o.cycletime - r.cycle_time_sec, 2) AS variance_cycle_time,
  ROUND(
    try_divide(
      o.approved_quantity * o.cycletime,
      o.actual_net_production_time
    ),
    4
  ) AS oee_estimate,
  r.setup_time_mins,
  r.labour_time_sec
FROM workspace.default.output o
JOIN workspace.default.routings r
  ON o.production_order = r.order_id
 AND o.material = r.material_id
 AND o.plant = r.plant_id
 AND o.line = r.line
JOIN routing_variants v
  ON r.material_id = v.material_id AND r.id = v.routing_id
ORDER BY o.material, o.plant, throughput_per_hour DESC;


material,production_order,plant,line,routing_id,linetype,order_start_ts,order_end_ts,duration_days,approved_quantity,actual_net_production_time,throughput_per_hour,actual_cycle_time,planned_cycle_time,variance_cycle_time,oee_estimate,setup_time_mins,labour_time_sec
4802569,285329,2,72,1,203,2025-01-07T23:28:51.000Z,2025-01-08T12:37:49.000Z,1,1119780,1035900,3891.5,14.048999999999982,14.049,0.0,15.1866,28,14.049
4802569,330019,3,50,1,103B,2025-01-01T07:42:32.000Z,2025-01-01T18:17:14.000Z,0,628480,589080,3840.78,11.772000000000007,11.772,0.0,12.5594,28,11.772
4806819,229097,2,26,1,B204,2025-01-02T09:55:43.000Z,2025-01-04T09:07:43.000Z,2,19531598,14191444,4954.66,11.367999999999817,11.368,0.0,15.6457,59,11.368
4806819,335517,3,39,1,102B,2025-01-09T09:14:31.000Z,2025-01-10T01:28:46.000Z,1,1616030,1369487,4248.09,7.856000000000014,7.856,0.0,9.2703,46,9.427
4828358,358644,1,67,1,2016,2025-01-14T00:16:02.000Z,2025-01-15T06:39:22.000Z,1,666678,2765313,867.91,11.612999999999875,11.613,0.0,2.7997,46,13.935
4828358,284243,2,25,1,2501,2025-01-07T01:23:11.000Z,2025-01-07T14:36:59.000Z,0,316867,1087040,1049.38,12.414000000000009,12.414,0.0,3.6186,40,12.414
4828358,284191,2,25,1,2501,2025-01-10T00:34:43.000Z,2025-01-12T09:37:50.000Z,2,5797484,25695824,812.23,12.413999999999374,12.414,0.0,2.8008,40,12.414
4828358,284250,2,25,1,2501,2025-01-07T17:29:45.000Z,2025-01-10T00:34:13.000Z,3,5238613,23835594,791.21,12.414000000000025,12.414,0.0,2.7284,40,12.414
4840351,277484,1,31,3,2016,2025-01-10T19:29:53.000Z,2025-01-12T10:19:27.000Z,2,23356116,13264902,6338.68,11.076999999999831,11.077,0.0,19.5038,46,11.077
4840351,308189,2,1,3,A408,2025-01-02T06:47:48.000Z,2025-01-02T16:00:44.000Z,0,1116192,725396,5539.44,9.03999999999998,9.04,0.0,13.9102,40,9.04


## ✅ Summary of Findings

- Several materials are produced in multiple plants using the same routing ID, yet their setup and labour times differ significantly.
- These setup differences correlate with measurable changes in **throughput per hour** and overall production efficiency.
- In many cases, plants with **lower setup times** and **labour inputs** achieve **higher throughput**, even when cycle times remain consistent.
- Analysis of `order_start_ts` and `order_end_ts` reveals variation in execution duration, suggesting that some plants may experience more downtime or scheduling inefficiencies.

### 📌 Key Insight:
Routing ID alone does not guarantee consistent performance. Setup parameters and plant-level execution practices play a critical role in determining output efficiency.

### 📌 Recommendation:
Standardize routing setups for high-volume materials and benchmark plant performance to identify best practices that can be scaled across the network.


Interpreting the OEE Values
This is a performance × quality approximation, assuming availability is 100% (since downtime isn’t tracked here).
